## 리그 오브 레전드 리그별 경기 결과 페이지 url 크롤링
  
https://qwer.gg/ko/leagues/{league}/{year}/{season}

+ 리그 : LCK(한국), LPL(중국), LCS(북미), LES(유럽)
+ 연도 : 2015 ~
+ 시즌 : 스프링, 서머

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import pandas as pd
import time
import os
import warnings
warnings.filterwarnings("ignore")

In [3]:
options = webdriver.ChromeOptions()
#options.headless = True
options.add_argument('window_size=1920x1080')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36')

browser = webdriver.Chrome(options=options)

#qwer 사이트에서 LCK 2015년 서머시즌 페이지 열기
league = 'LCK'
year = '2015'
season = 'Summer'
url = f'https://qwer.gg/ko/leagues/{league}/{year}/{season}'

browser.get(url)

In [ ]:
#시즌 내 경기타입 선택
button = browser.find_element(By.ID, 'headlessui-menu-button-16')
button.click()

season_types = browser.find_element(By.ID, 'headlessui-menu-items-17').find_elements(By.TAG_NAME, 'div')
print(season_types[1].get_attribute('innerText'))

season_types[0].click()

Regular season


In [4]:
#경기결과 클릭
button = browser.find_element(By.ID, 'headlessui-tabs-tab-10')
button.click()   
time.sleep(3)

In [ ]:
#각 매치 결과 페이지 href 크롤링
buttons = browser.find_elements(By.TAG_NAME, 'button')

for i in reversed(buttons[16:]):   #매치 별 detail 버튼 누르기
    i.click()
    
a_tag = browser.find_elements(By.TAG_NAME, 'a')
match_link = [i.get_attribute('href') for i in a_tag][17:-5]

#txt 파일에 저장
filename = f'{league}_{year}_{season}_match_links.txt'
with open(filename, 'w') as file:
    file.write('\n'.join(match_link))

In [ ]:
#위 과정을 함수로 만들기
def get_match_url(league, year, season):
    for type_idx in range(4):
        options = webdriver.ChromeOptions()
        options.headless = True   #브라우저 창 띄우지 않고 백그라운드에서 진행
        options.add_argument('window_size=1920x1080')
        options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36')
        options.add_experimental_option("detach", True)
        browser = webdriver.Chrome(options=options)

        url = f'https://qwer.gg/ko/leagues/{league}/{year}/{season}'
        
        try:
            browser.get(url)
            time.sleep(3)
            
            button = browser.find_element(By.ID, 'headlessui-menu-button-16')  #상세시즌 사이드바
            button.click()
            
            season_types = browser.find_element(By.ID, 'headlessui-menu-items-17').find_elements(By.TAG_NAME, 'div')   #상세시즌 내역
                    
            season_type = season_types[type_idx]     
            season_name = season_type.get_attribute('innerText')  #txt 파일 저장 시 이름에 사용
                        
            season_type.click()
                
            button = browser.find_element(By.ID, 'headlessui-tabs-tab-10')
            button.click()   
            time.sleep(1)
                    
            buttons = browser.find_elements(By.TAG_NAME, 'button')

            for i in reversed(buttons[16:]):   #매치 별 detail 버튼 누르기
                i.click()
            time.sleep(1)
                    
            a_tag = browser.find_elements(By.TAG_NAME, 'a')
            match_link = [i.get_attribute('href') for i in a_tag][17:-5]

            #txt 파일에 저장
            filename = f'{league}_{year}_{season}_{season_name}_match_links.txt'
            with open(filename, 'w') as file:
                file.write('\n'.join(match_link))
            time.sleep(1)
            browser.quit()
        
        except:
            browser.quit()    

In [ ]:
#약 80분 소요
leagues = ['LCK', 'LPL', 'LCS', 'LEC']
years = [str(i) for i in range(2015, 2023)]
seasons = ['Spring', 'Summer']

for league in leagues:
    for year in years:
        for season in seasons:
            get_match_url(league, year, season)

In [1]:
#폴더 내 txt 파일 모두 불러오는 함수
def get_txt_files(path):
    txt_file_list = [file for file in os.listdir(path) if file.endswith('txt')]
    return(txt_file_list)
    
txt_list = get_txt_files('./')    #현재 디렉토리 내 txt 파일들(116개)

txt_list

['LCK_2015_Summer_Playoffs_match_links.txt',
 'LCK_2015_Summer_Promotion_match_links.txt',
 'LCK_2015_Summer_Regular season_match_links.txt',
 'LCK_2016_Spring_Playoffs_match_links.txt',
 'LCK_2016_Spring_Promotion_match_links.txt',
 'LCK_2016_Spring_Regular season_match_links.txt',
 'LCK_2016_Summer_match_links.txt',
 'LCK_2016_Summer_Playoffs_match_links.txt',
 'LCK_2016_Summer_Promotion relegation_match_links.txt',
 'LCK_2016_Summer_Regionals_match_links.txt',
 'LCK_2016_Summer_Regular season_match_links.txt',
 'LCK_2017_Spring_Playoffs_match_links.txt',
 'LCK_2017_Spring_Promotion relegation_match_links.txt',
 'LCK_2017_Spring_Regular season_match_links.txt',
 'LCK_2017_Summer_Playoffs_match_links.txt',
 'LCK_2017_Summer_Promotion relegation_match_links.txt',
 'LCK_2017_Summer_Regionals_match_links.txt',
 'LCK_2017_Summer_Regular season_match_links.txt',
 'LCK_2018_Spring_Playoffs_match_links.txt',
 'LCK_2018_Spring_Promotion relegation_match_links.txt',
 'LCK_2018_Spring_Regular s